In [ ]:
''' Install dependencies '''
#pip install pypdf
#pip install langchain langchain_community langchain_core langchain_openai langchain_text_splitters
#pip install -qU langchain-chroma
#pip install langchainhub
#pip install -U "langchain[anthropic]"
#pip install -qU langchain-voyageai

In [4]:
''' Load Claude AI API key from .env '''
import os
import dotenv
from dotenv import load_dotenv
import getpass

load_dotenv()
#os.environ["ANTHROPIC_API_KEY"] = os.getenv("ANTHROPIC_API_KEY")
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

''' Load Voyage AI API key from .env '''
os.environ["VOYAGE_API_KEY"] = os.getenv("VOYAGE_API_KEY")

''' Load Langsmith API key from .env '''
os.environ["LANGSMITH_TRACING"] = "true"
if not os.environ.get("VOYAGE_API_KEY"):
    os.environ["VOYAGE_API_KEY"] = getpass.getpass("Enter API key for Voyage AI: ")


In [2]:
''' Set up necessary libraries and environment '''
import bs4
import langchainhub
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_anthropic import ChatAnthropic
from langchain_voyageai import VoyageAIEmbeddings

c:\Users\chery\projects\insurance-policy-rag\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
''' Set up document loader '''
from langchain_community.document_loaders import PyPDFLoader #alternative: PyMuPDFLoader
from langchain_community.document_loaders import PyMuPDFLoader #alternative: PyMuPDFLoader

policy_pdf = os.getenv("PDF_PATH")

loader = PyMuPDFLoader(policy_pdf, extract_tables_settings={"enabled": True})

documents = loader.load()

MuPDF error: library error: zlib error: incorrect header check

MuPDF error: library error: zlib error: incorrect header check

MuPDF error: library error: zlib error: incorrect header check

MuPDF error: library error: zlib error: incorrect header check

MuPDF error: library error: zlib error: incorrect header check

MuPDF error: library error: zlib error: incorrect header check

MuPDF error: library error: zlib error: incorrect header check

MuPDF error: library error: zlib error: incorrect header check

MuPDF error: library error: zlib error: incorrect header check

MuPDF error: library error: zlib error: incorrect header check

MuPDF error: library error: zlib error: incorrect header check

MuPDF error: library error: zlib error: incorrect header check

MuPDF error: library error: zlib error: incorrect header check

MuPDF error: library error: zlib error: incorrect header check

MuPDF error: library error: zlib error: incorrect header check



In [ ]:
''' Redact sensitive information from documents (PII)'''
'''
import re
from langchain_core.documents import Document

policy_number = input("Enter your policy number (used only for one-time redaction): ").strip()
redaction_patterns = {
    "email": r"\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}\b",
    "government_id": r"\b\d{6}-\d{2}-\d{4}\b",  # 123456-78-9000
}

def redact_sensitive_info(text, policy_number):
    redacted = text
    for pattern_name, pattern in redaction_patterns.items():
        redacted = re.sub(pattern, f"[Redacted_{pattern_name.upper()}]", redacted)
    redacted = re.sub(re.escape(policy_number), "[Redacted_policy_number]", redacted)
    return redacted

redacted_documents = []

for doc in documents: 
    redacted_content = redact_sensitive_info(doc.page_content, policy_number=policy_number)
    redacted_documents.append(
        Document(page_content=redacted_content))
    
documents = redacted_documents
'''

In [6]:
documents[0].page_content[:500]  

''

In [7]:
print("Number of pages loaded:", len(documents))
print("Page 1 text snippet:\n", documents[0].page_content[:1000])

Number of pages loaded: 15
Page 1 text snippet:
 


In [ ]:
''' Split text, create vectorstore and retriever '''

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(documents)

vectorstore = Chroma.from_documents(documents=splits, embedding=VoyageEmbeddings(model="voyage-2"))
retriever = vectorstore.as_retriever()

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [ ]:
from langchain_anthropic import ChatAnthropic

model = ChatAnthropic(model="claude-sonnet-4-5-20250929")

In [ ]:
prompt = langchainhub.pull("rlm/rag-prompt")

'''
You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: {question}
Context: {context}
Answer:
'''

In [ ]:
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

In [ ]:
def format_docs(docs):
    return "\\n\\n".join(doc.page_content for doc in docs)

In [ ]:
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
result = rag_chain.invoke("What is Onepane pixie connector used for?")
print(result)

In [ ]:
vectorstore.delete_collection()